In [1]:
import pandas as pd
from whoosh.index import create_in
from whoosh.fields import TEXT, SchemaClass
from jieba.analyse import ChineseAnalyzer
import jieba
from whoosh.qparser import QueryParser, MultifieldParser
from whoosh.index import open_dir
import re
import hashlib
from openpyxl import load_workbook

In [2]:
analyzer = ChineseAnalyzer()
class TextSchema(SchemaClass):
    title = TEXT(stored=True, analyzer=analyzer)
    content = TEXT(stored=True, analyzer=analyzer)

In [3]:
def clean_text(text):
    # 去掉所有非字母字符
    text = re.sub(r'[^A-Za-z\u4e00-\u9fa5]', '', text)
    # 把所有英文单词转为大写
    text = text.upper()
    return text

In [4]:
# 读取Excel文件
df = pd.read_excel('test_data.xlsx')
df = df.dropna()
for i in ['title','content_clean']:
    df[i] = df[i].apply(clean_text)
df = df.drop_duplicates(subset='content_clean')
df

,title,content_clean,brandName_list,待标注关键词
0,谁还不是精致女孩呢,今天是护肤日明天就要健身啦SILKN真的是我唯一的美容仪器还坚持用下来了因为凝胶快过期而提高...,['silk’n_美容仪'],护肤，拯救干燥肌肤，精致
1,宙斯反馈,去年买的宙斯泪沟没了自己都不敢相信使用率可能想起来连着做几天忘了就半个多月不做既然有这个功效...,['Dr.Arrivo_美容仪'],泪沟消失，功效显著，坚持使用，改善肤质，方便易用，持续效果，抗衰
2,雅萌好物开箱,年纪过了肌肤状态明显开始走下坡路挣扎了好久终于狠心入手了雅萌美容仪收到包裹后开箱过程都很享受...,['雅萌YA-MAN_美容仪'],护肤，精致大气，舒适，修复，持续使用
3,想买美容仪拯救皮肤了宙斯二代美容仪怎么,求介绍以前化妆总是很应付卸妆更是比较随意有时候赶紧比较累直接带妆睡觉结果导致皮肤越来越粗糙化...,['Dr.Arrivo_美容仪'],多档位调节，舒适，温和，消肿，宙斯二代，效果显著
4,过了岁别再瞎买美容仪了直接上初普吧,实话实说射频咱就得服人家以色列妥妥射频届的老大哥我人生的第一台美容仪就是TRIPOLLAR初...,['TriPollar_美容仪'],射频美容仪，安全，资深，升级技术
...,...,...,...,...
6874,留学公演ROUTINE大公开,人人都可以站上舞台的嘞果姐的公演从开始到结束都大大公开顺便分享一些好东西给大家维密反重力小升...,"[""维多利亚的秘密Victoria's Secret_内衣""]",维密，反重力，小升杯，显身材
6875,春日出游穿搭分享,出片率的秘密五一快到啦大家有想好去哪玩嘛感觉每次出门都不知道穿什么真的很苦恼今天这期分享了一...,['蕉内Bananain_内衣'],好的搭配，五一出行穿搭。
6876,爱慕爱生活去繁存真内在本真,大家节日快乐呀这是节我送给自己的一份精美礼物爱慕三十周年礼盒一套内衣真丝眼罩笔记本礼盒包装精...,['爱慕Aimer_内衣'],外穿内衣，清爽舒适，无钢圈支撑，V领无托，玫瑰花设计，爱慕30周年纪念
6877,解开束缚女孩子都是自由肆意的,HELLO宝贝们我是麻花今天男士止步女士请进敲咪咪来给姐妹们分享一下我的宝藏内衣爱美丽这一件...,"[""爱美丽imi's_内衣""]",夹心软支撑 稳定有型 柔软 手感光滑 有弹性 轻盈透气 无勒痕


In [8]:
#query title&content
schema = TextSchema()
ix = create_in("indexdir", schema, indexname='text_index')
writer = ix.writer()
for i in range(int(len(df))):
    writer.add_document(title=df.iloc[i,0],content=df.iloc[i,1])
writer.commit()

In [9]:
#query keywords
schema = TextSchema()
ix = create_in("indexdirkey", schema, indexname='text_index')
writer = ix.writer()
for i in range(int(len(df))):
    writer.add_document(title=df.iloc[i,0],content=df.iloc[i,1])
writer.commit()


In [10]:
def make_query(brandName_list , key_word):
    query_list = []
    tem_list = brandName_list[2:-2].split("_")
    string = tem_list[0]
    english_chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
    # 将字符串拆分为中英文两部分
    chinese_part, english_part = "", ""
    for char in string:
        if char in english_chars:
            english_part += char
        else:
            chinese_part += char
    tem_list.append(tem_list[1])
    tem_list[0] = chinese_part
    tem_list[1] = english_part
    for i in tem_list:
        query_list.append(i)
    tem_list = key_word.split("，")
    for i in tem_list:
        query_list.append(i)
    for i in range(len(query_list)):
        tem = clean_text(query_list[i])
        query_list[i] = tem
    return query_list

In [11]:
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
def write_dicts_to_xlsx(data, filename):
    # 创建一个新的工作簿
    workbook = Workbook()
    sheet = workbook.active
    # 写入数据
    index = 0
    for dict_data in data:
        # 写入一个空行
        tem = [df.iloc[index,0],df.iloc[index,1],df.iloc[index,2],df.iloc[index,3]]
        sheet.append(tem)
        index = index + 1
        # 写入字典数据
        count = 0
        for key, value in dict_data.items():
            sheet.append([key, value])
            count += 1
            if count == 10:
                break
    # 保存工作簿到文件
    workbook.save(filename)

In [16]:
#fix = open_dir("indexdir", indexname='text_index')
fix = open_dir("indexdirkey", indexname='text_index')

with fix.searcher() as searcher:
    keyword_hashes = []
    for i in range(int(len(df))):
    #for i in range(1):
        query_list  = make_query(df.iloc[i,2],df.iloc[i,3])
        keyword_counts = {}
        index = 0
        for j in query_list:
            query = MultifieldParser(["title","content"], fix.schema).parse(j)
            results = searcher.search(query)
            for data in results:
                word = data['content']
                if word in keyword_counts:
                    if index < 2:
                        if j =='美容仪' or j =='智能门锁'or j =='益生菌'or j =='内衣':
                            keyword_counts[word] += 3
                        else:
                            keyword_counts[word] += 5
                    elif j =='美容仪' or j =='智能门锁'or j =='益生菌'or j =='内衣':
                        keyword_counts[word] += 3
                    else:
                        keyword_counts[word] += 1
                else:
                    if index < 2:
                        if j =='美容仪' or j =='智能门锁'or j =='益生菌'or j =='内衣':
                            keyword_counts[word] = 3
                        else:
                            keyword_counts[word] = 5
                    elif j =='美容仪' or j =='智能门锁'or j =='益生菌'or j =='内衣':
                        keyword_counts[word] = 3
                    else:
                        keyword_counts[word] = 1
            index += 1
        sorted_keyword_counts = sorted(keyword_counts.items(), key=lambda x: x[1], reverse=True)
        # 创建一个新的字典以保持原字典不变
        sorted_dict = dict(sorted_keyword_counts)
        keyword_hashes.append(sorted_dict)
    filename = 'output_keyword.xlsx'
    write_dicts_to_xlsx(keyword_hashes, filename)

In [2]:
# 读取Excel文件
df_result = pd.read_excel('panguKeyword_output.xlsx', header=None)
df_result

,0,1,2,3,4,5,6
0,[雅萌YA-MAN],美容仪,[美女],大美女张嘉倪、雅萌ACE、种草、美容仪器、租、平价护肤、亲测好用,明星同款的雅萌ACE美容仪唉,大美女张嘉倪同款的雅萌ACE已经种草很久了，一直舍不得买，几千块太贵了，这两天发现了美容仪器...,NaN
1,[雅萌YA-MAN],美容仪,[美女],大美女张嘉倪、雅萌ACE、种草、美容仪器、租、平价护肤、亲测好用,明星同款的雅萌ACE美容仪唉,大美女张嘉倪同款的雅萌ACE已经种草很久了，一直舍不得买，几千块太贵了，这两天发现了美容仪器...,0.998441
2,[雅萌YA-MAN],美容仪,[美女],大美女张嘉倪同款、雅萌ACE、种草、家用仪器、租赁、平价护肤、亲测好用美容仪,行动起来！女明星同款买不起还租不起了？,大美女张嘉倪同款的雅萌ACE已经种草很久了，一直舍不得买，几千块太贵了，这两天发现了美容仪器...,0.983659
3,[雅萌YA-MAN],美容仪,[美女],大美女张嘉倪、同款、雅萌ACE、种草、美容仪器、租、平价护肤、好用美容仪、实测,女明星同款买不起还租不起了？嘿嘿,大美女张嘉倪同款的雅萌ACE已经种草很久了，一直舍不得买，几千块太贵了，这两天发现了美容仪器...,0.976570
4,[雅萌YA-MAN],美容仪,[美女],大美女张嘉倪、种草、美容仪器、租、学生党、生活费、平价护肤、租赁、亲测好用美容仪,女明星同款买不起还租不起了？,大美女张嘉倪同款的雅萌ACE已经种草很久了，一直舍不得买，几千块太贵了，这两天发现了美容仪器...,0.934816
...,...,...,...,...,...,...,...
6991,[WonderLab],益生菌,"[小孩, 宝妈]",宝妈、放纵吃喝、底气、胡吃海喝、厕所、菌群、好菌、益生菌、菌株、活性、肠胃、安全、耐酸耐胆汁...,宝妈放纵吃喝的底气❗有它在随时保持状态💯,宝妈放纵吃喝的底气❗有它在随时保持状态💯🚨一出去胡吃海喝就跑厕所❓\n可能是受菌群影响⚠️\...,0.999492
6992,[WonderLab],益生菌,"[儿童, 姐妹, 宝妈]","WonderLab, 小黄瓶益生菌, 儿童益生菌, 母乳菌, 肠道, 宝宝吸收, 四种菌群科...",白白叶叶＆ wanderlab 益生菌专场来了,1、WonderLab 小黄瓶益生菌\n宝宝肠胃不好⁉️换季总生病⁉️建议宝妈们人手备一盒W...,0.824872
6993,[拜耳BAYER],益生菌,"[孩子, 宝妈]",益生菌、温和不刺激、爽、减肥、燃动菌、中国注册专利菌株、伊能静同款、敏感肠胃、胡吃海喝、均衡...,居家必备单品之达益喜,居家必备单品之达益喜 生孩子以后就开始便秘了 也不知道是不是因为顺产的原因😭 便秘严重的时候...,0.816018
6994,[WonderLab],益生菌,"[妈妈, 老爸, 儿童]",崔玉涛力荐、宝宝肠胃道益生元、调理肠道、胃口好、酸奶口味、柔滑可口、混合益生菌、改善消化不良...,买wonderlab小黄瓶益生菌送滑板车啦,老爸抽检崔玉涛力荐，万益蓝wonderlab儿童即食益生菌宝宝肠胃道益生元，给宝宝一个快乐的...,0.812889


In [14]:
case3 = 0
case5 = 0
case8 = 0
sum_case = 0
for i in range(int(len(df_result))):
    if i%11 != 0:
        sum_case = sum_case + 1
        if df_result.iloc[i,1] <= 3:
            case3 +=1
        if df_result.iloc[i,1] <= 5:
            case5 +=1
        if df_result.iloc[i,1] <= 8:
            case8 +=1
            
print(case3 / sum_case , case5 / sum_case ,case8 / sum_case )

0.026465335281701575 0.8780970936840397 0.9644422795908427


In [24]:
# 读取Excel文件
df_result = pd.read_excel('panguKeyword_output.xlsx', header=None)
df_result

,0,1,2,3,4
0,[雅萌YA-MAN],美容仪,[美女],大美女张嘉倪、雅萌ACE、种草、美容仪器、租、平价护肤、亲测好用,NaN
1,[雅萌YA-MAN],美容仪,[美女],大美女张嘉倪、雅萌ACE、种草、美容仪器、租、平价护肤、亲测好用,0.998441
2,[雅萌YA-MAN],美容仪,[姐妹],NaN,0.811628
3,[雅萌YA-MAN],美容仪,[姐妹],官网购买、用了四五次、喜欢的姐妹、美容仪、射频美容仪、家用美容仪、雅萌MAX二代、出闲置、雅萌,0.797212
4,[雅萌YA-MAN],美容仪,NaN,618、仪器、凝胶、闲置、用不到、家用、雅萌、美容仪、ACE,0.785518
...,...,...,...,...,...
3295,[卡尔文·克莱恩Calvin Klein],内衣,[姐妹],h杯、感动、其他牌子、大罩杯、女大、ck、维秘、舒适内衣推荐、大胸内衣,0.743446
3296,[卡尔文·克莱恩Calvin Klein],内衣,NaN,甜酷辣妹风、轻松get、舒适度、性价比、心动、同款、ootd每日穿搭、舒适内衣、拍照姿势、ck,0.742837
3297,[卡尔文·克莱恩Calvin Klein],内衣,"[女孩, 姐妹]",运动、折扣、太阳城金字塔商场、Calvin Klein、40%OFF、搭配、时尚、Sunwa...,0.740574
3298,[卡尔文·克莱恩Calvin Klein],内衣,NaN,NaN,0.739463


In [25]:
for i in range(int(len(df_result))):
    if i%11 >5 :
        df_result = df_result.drop(i)
df_result[0:20]

,0,1,2,3,4
0,[雅萌YA-MAN],美容仪,[美女],大美女张嘉倪、雅萌ACE、种草、美容仪器、租、平价护肤、亲测好用,NaN
1,[雅萌YA-MAN],美容仪,[美女],大美女张嘉倪、雅萌ACE、种草、美容仪器、租、平价护肤、亲测好用,0.998441
2,[雅萌YA-MAN],美容仪,[姐妹],NaN,0.811628
3,[雅萌YA-MAN],美容仪,[姐妹],官网购买、用了四五次、喜欢的姐妹、美容仪、射频美容仪、家用美容仪、雅萌MAX二代、出闲置、雅萌,0.797212
4,[雅萌YA-MAN],美容仪,NaN,618、仪器、凝胶、闲置、用不到、家用、雅萌、美容仪、ACE,0.785518
5,[雅萌YA-MAN],美容仪,[姐妹],抗老、双十一、美容仪推荐、seeme射频仪仪、极萌美容仪、雅萌美容仪、花至美容仪、射频仪、祛...,0.774465
11,[卡尔文·克莱恩Calvin Klein],内衣,NaN,棋盘格设计、时尚风格、XELLA系列、清爽自然、棉触感、轻便舒适、宽肩带、稳定性、方领设计、...,NaN
12,[卡尔文·克莱恩Calvin Klein],内衣,NaN,棋盘格设计、时尚风格、XELLA系列、清爽自然、棉触感、轻便舒适、宽肩带、稳定性、方领设计、...,0.999586
13,[卡尔文·克莱恩Calvin Klein],内衣,[明星],CK时尚配色、运动系列、穿搭需求、摩登时尚、棉质系列、欧美风、三角杯、工字背心、一体式肩部开...,0.871803
14,[卡尔文·克莱恩Calvin Klein],内衣,NaN,精心打扮、美美、法式、简约经典、白色、缎面、贵气、A字型、沙漏式身材、纯色、耐看、高级、优雅...,0.847323


In [3]:
good_case = 0
right_case = 0
sum_case = 0
for i in range(int(len(df_result))):
    if i%6 == 0:
        title = df_result.iloc[i,0]
        type_name = df_result.iloc[i,1]
    else:
        sum_case = sum_case + 1
        if title == df_result.iloc[i,0]:
            good_case = good_case + 1
        if type_name == df_result.iloc[i,1]:
            right_case = right_case + 1
        else:
            print(i)
            
print(good_case / sum_case , right_case / sum_case )

1005
1059
1060
1061
1282
3635
4595
5931
6358
6590
6591
6593
6788
6790
0.9068610634648371 0.997598627787307


In [30]:
df_duibi = pd.read_excel('pangu_result-no-should-top5.xlsx')
df_duibi[0:20]

,score,title,content,brand,category,feature,crowd,ctime,type
0,0.000000,明星同款的雅萌ACE美容仪唉,大美女张嘉倪同款的雅萌ACE已经种草很久了，一直舍不得买，几千块太贵了，这两天发现了美容仪器...,雅萌YA-MAN,美容仪,大美女张嘉倪、雅萌ACE、种草、美容仪器、租、平价护肤、亲测好用,美女,2023-11-08,query
1,3.382977,明星同款的雅萌ACE美容仪唉,大美女张嘉倪同款的雅萌ACE已经种草很久了，一直舍不得买，几千块太贵了，这两天发现了美容仪器...,雅萌YA-MAN,美容仪,大美女张嘉倪、雅萌ACE、种草、美容仪器、租、平价护肤、亲测好用,美女,2023-11-08,result
2,3.264468,1999！极萌 胶原炮Max熨斗美容仪多少米,谁懂我对极萌的爱？？ 只要一出新品我都马不停蹄的拿下 说真的无论它家是做脱毛仪还是美容仪 用...,雅萌YA-MAN,美容仪,极萌、美容仪、家用美容仪、雅萌美容仪、不踩雷、美容院、踩雷,NaN,2023-11-08,result
3,3.254097,雅萌美容仪值得购买,#美容仪[话题]# 这是我第一次买这么贵的美容仪，雅萌是我关注了两年才买的昨天刚到打开我就给...,雅萌YA-MAN,美容仪,雅萌、美容仪、全套模式、震动感、脸部提亮,老公,2023-11-08,result
4,3.211926,雅萌二代带你逆龄飞升！,亲爱的小仙女们✨，我又来分享我的新发现啦！这次，我要告诉你们的是一款让我完全放飞自我的美容仪...,雅萌YA-MAN,美容仪,小仙女、简约大方、焕模、EMS提升模式、大优惠、逆龄飞升、美丽选择、护肤、惊喜、京东家电,NaN,2023-11-08,result
5,3.207041,还不会选美容仪？快进来看！,我也算是美容仪大户了\n从几年前就开始用美容仪\n亲身使用这么多年，我算是有发言权的！\n美...,雅萌YA-MAN,美容仪,美容仪、智商税、真皮层、性价比、护肤、抗老,年轻,2023-11-08,result
6,0.000000,吸晴棋盘格Bra 夏日里的一抹活力🌈,厌倦了纯色BRA❓\n来看XELLA INTENTION CHECK BRA TOP\n「X...,卡尔文·克莱恩Calvin Klein,内衣,棋盘格设计、时尚风格、XELLA系列、清爽自然、棉触感、轻便舒适、宽肩带、稳定性、方领设计、...,NaN,2023-11-08,query
7,2.966535,吸晴棋盘格Bra 夏日里的一抹活力🌈,厌倦了纯色BRA❓\n来看XELLA INTENTION CHECK BRA TOP\n「X...,卡尔文·克莱恩Calvin Klein,内衣,棋盘格设计、时尚风格、XELLA系列、清爽自然、棉触感、轻便舒适、宽肩带、稳定性、方领设计、...,NaN,2023-11-08,result
8,2.704045,看过来喽！CK时尚配色，爱到脱不下来❤️,看过来喽！CK时尚配色，爱到脱不下来❤️CK运动系列，丰富您的穿搭需求|||\n摩登时尚棉质...,卡尔文·克莱恩Calvin Klein,内衣,CK时尚配色、运动系列、穿搭需求、摩登时尚、棉质系列、欧美风、三角杯、工字背心、一体式肩部开...,明星,2023-11-08,result
9,2.651768,应该没有人能拒绝ck的魅力吧！！,甜酷辣妺风轻松get 除了颜值之外还很舒服 性价比又高 怎能不心动呢 👙同款：@小熊的店 #...,卡尔文·克莱恩Calvin Klein,内衣,甜酷辣妹风、轻松get、舒适度、性价比、心动、同款、ootd每日穿搭、舒适内衣、拍照姿势、ck,NaN,2023-11-08,result


In [33]:
df_duibi.iloc[0,3]

'雅萌YA-MAN'

In [39]:
good_case = 0
right_case = 0
sum_case = 0
for i in range(int(len(df_duibi))):
    if df_duibi.iloc[i,8] =='query' :
        title = df_duibi.iloc[i,3]
        type_name = df_duibi.iloc[i,4]
    else:
        sum_case = sum_case + 1
        if title == df_duibi.iloc[i,3]:
            good_case = good_case + 1
        if type_name == df_duibi.iloc[i,4]:
            right_case = right_case + 1
        else:
            print(i)
    #print(i,title,df_result.iloc[i,3],type_name,df_result.iloc[i,4])        
print(good_case / sum_case , right_case / sum_case )

0.7270029673590505 1.0


In [12]:
import json
with open('pangu_test.json', 'r') as f:
    lines = f.readlines()
data = []
for line in lines:
    try:
        data_for_line = json.loads(line)
        # 如果解析成功，则添加到数据列表中
        data.append(data_for_line)
    except json.JSONDecodeError:
        # 打印出无法解析的行并跳过
        print(f"Skipping invalid JSON line: {line}")
data[0:4]

[{'dwd_content_id': '0158df647815a1c308446e40a19458772',
  'type': 2,
  'source': '1027',
  'is_hot': 2,
  'title': '明星同款的雅萌ACE美容仪唉',
  'content': '大美女张嘉倪同款的雅萌ACE已经种草很久了，一直舍不得买，几千块太贵了，这两天发现了美容仪器还能租，之前看朋友买心痒痒但是又怕买回来效果不好毕竟也不是小钱纠结死了，有人租过吗？怎么样呀？ #租美容仪#雅萌#平价护肤#亲测好用美容仪#雅萌ACE#租赁',
  'is_emoji': 2,
  'brand_name_list': '[雅萌YA-MAN]',
  'brand_id_list': '[雅萌YA-MAN]',
  'category_name_list': '[美容仪]',
  'category_id_list': '[2730600]',
  'brand_keyword': '[美女, 平价, 不好, 亲测, 种草, 太贵]',
  'content_pred': -1,
  'share_url': 'https://www.xiaohongshu.com/discovery/item/6524c662000000001f036189',
  'scenes_keyword_list': '',
  'content_style': '["彩妆护肤"]',
  'crowd_keyword_list': '[美女]',
  'baichuan2_keyword': '营销产品：雅萌ACE\n关键词：大美女张嘉倪、雅萌ACE、种草、美容仪器、租、平价护肤、亲测好用',
  'baichuan2_keyword_clean': '大美女张嘉倪、雅萌ACE、种草、美容仪器、租、平价护肤、亲测好用',
  'brand_3class': '美容仪',
  'dim_keyword_level3_list': '[性别, 价格评价, 评测榜单, 产品推荐, 差评]',
  'dim_keyword_level4_list': '[有性价比的, 女性, 太贵的]'},
 {'dwd_content_id': '020c321ce62ef3662f98021c53d495